In [ ]:
import json
import numpy as np

dataset_name_ls = [
    "adult",
    "lung_cancer",
    "cylinder-bands",
    "diabetes",
    "contraceptive",
    "blood-transfusion-service-center",
    "steel-plates-fault",
    "jungle_chess_2pcs_raw_endgame_complete",
    "telco_customer_churn",
    "bank-marketing",
    "PhishingWebsites",
    "hotel_bookings",
]
mode_ls = ["gt", "ignore", "coerce", "rectify"]
sql_ls = ["q1.sql", "q2.sql", "q3.sql", "q4.sql"]

result_dict_full = {}  # dataset_name -> mode-> result
for dataset_name in dataset_name_ls:
    with open(f"../example_query/{dataset_name}/rq2_result.json", "r") as f:
        rq2_result = json.load(f)
    result_dict = {}
    possible_invalid_keys = {}
    for mode in mode_ls:
        result = []
        for sql_name in sql_ls:
            gt_result = np.array(rq2_result[sql_name + ",gt"])
            cur_result = np.array(rq2_result[sql_name + "," + mode])
            if gt_result.shape[0] != cur_result.shape[0]:
                if gt_result.shape[1] > 2:
                    print("error")
                    print(dataset_name, mode, sql_name)
                if mode == "coerce":
                    vaild_result = np.zeros(gt_result.shape[0])
                    for i in range(len(cur_result)):
                        print(np.where(gt_result[:, 0] == cur_result[i][0]))
                        row_index = np.where(gt_result[:, 0] == cur_result[i][0])[0][0]
                        vaild_result[row_index] = cur_result[i][1]
                    result.extend(vaild_result.astype(float))
                else:
                    print("error")
                    print(dataset_name, mode, sql_name)
            else:
                if gt_result.shape[1] > 1:
                    if not np.array_equal(gt_result[:, 0], cur_result[:, 0]):
                        print("error")
                        print(dataset_name, mode, sql_name)
                        vaild_result = np.zeros(gt_result.shape[0])
                        for i in range(len(cur_result)):
                            invalid_idx = 0
                            if cur_result[i][0] in gt_result[:, 0]:
                                row_index = np.where(
                                    gt_result[:, 0] == cur_result[i][0]
                                )[0][0]
                                vaild_result[row_index] = cur_result[i][1]
                        result.extend(vaild_result.astype(float))
                    else:
                        vaild_result = cur_result[:, -1].astype(float)
                        vaild_result[np.isnan(vaild_result)] = 0
                        result.extend(vaild_result)
                else:
                    vaild_result = cur_result[:, -1].astype(float)
                    vaild_result[np.isnan(vaild_result)] = 0
                    result.extend(vaild_result)
        result_dict[mode] = result
    result_dict_full[dataset_name] = result_dict

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import collections as matcoll
import math

dataset_name_ls = [
    "adult",
    "lung_cancer",
    "cylinder-bands",
    "diabetes",
    "contraceptive",
    "blood-transfusion-service-center",
    "steel-plates-fault",
    "jungle_chess_2pcs_raw_endgame_complete",
    "telco_customer_churn",
    "bank-marketing",
    "PhishingWebsites",
    "hotel_bookings",
]
dt_name_ls_full = [
    "Adult",
    "Lung Cancer",
    "Cylinder Bands",
    "Diabetes",
    "Contraceptive Method",
    "Blood Trans. Serv. Ctr.",
    "Steel Plates Faults",
    "Jungle Chess",
    "Telco Customer Churn",
    "Bank Marketing",
    "Phishing Websites",
    "Hotel Reservations",
]
dt_name_dict = dict(zip(dataset_name_ls, dt_name_ls_full))

mode_ls = ["ignore", "rectify"]
sql_ls = ["q1.sql", "q2.sql", "q3.sql", "q4.sql"]

result_dict_full = {}
fig, axs = plt.subplots(
    nrows=2, ncols=6, figsize=(13, 3), subplot_kw={"xticks": [], "yticks": []}
)
plt.subplots_adjust(wspace=-0.5)
error_ls_ignore = []
error_ls_rectify = []

dataset_idx = 0
for ax, dataset_name in zip(axs.flat, dataset_name_ls):
    labels = ["#1", "#2", "#3", "#4"]

    results_ls = {"ignore": [], "rectify": []}

    with open(f"../example_query/{dataset_name}/rq2_result.json", "r") as f:
        rq2_result = json.load(f)

    for mode in mode_ls:
        for sql_name in sql_ls:
            gt_result = np.array(rq2_result[sql_name + ",gt"])
            cur_result = np.array(rq2_result[sql_name + "," + mode])
            if gt_result.shape[1] > 1:
                if not np.array_equal(gt_result[:, 0], cur_result[:, 0]):
                    print("error")
                    print(dataset_name, mode, sql_name)
                    vaild_result = np.zeros(gt_result.shape[0])
                    for i in range(len(cur_result)):
                        invalid_idx = 0
                        if cur_result[i][0] in gt_result[:, 0]:
                            row_index = np.where(gt_result[:, 0] == cur_result[i][0])[
                                0
                            ][0]
                            vaild_result[row_index] = cur_result[i][1]
                    result.extend(vaild_result.astype(float))
                else:
                    vaild_result = cur_result[:, -1].astype(float)
                    vaild_result[np.isnan(vaild_result)] = 0
            else:
                vaild_result = cur_result[:, -1].astype(float)
                vaild_result[np.isnan(vaild_result)] = 0
            l1_error = np.linalg.norm(
                gt_result[:, -1].astype(float) - vaild_result, 1
            ) / np.linalg.norm(gt_result[:, -1].astype(float), 1)
            results_ls[mode].append(l1_error)

    x = np.arange(len(labels))  # the label locations

    error_ls_ignore.extend(results_ls["ignore"])
    error_ls_rectify.extend(results_ls["rectify"])

    lines = []
    min_error = np.min([np.min(results_ls["ignore"]), np.min(results_ls["rectify"])])
    max_error = np.max([np.max(results_ls["ignore"]), np.max(results_ls["rectify"])])
    normalized_ignore = (results_ls["ignore"] - min_error) / (max_error - min_error)
    normalized_rectify = (results_ls["rectify"] - min_error) / (max_error - min_error)
    for idx, label in enumerate(labels):
        lines.append([(idx, normalized_ignore[idx]), (idx, normalized_rectify[idx])])
    linecoll = matcoll.LineCollection(lines, colors="black", linewidths=1)
    ax.add_collection(linecoll)
    ax.set_title(dt_name_dict[dataset_name], fontsize=11.3)

    ax.scatter(
        x, normalized_ignore, marker="o", color="red", label="w/o Guardrail", s=11
    )
    ax.scatter(
        x, normalized_rectify, marker="*", color="blue", label="w/ Guardrail", s=11
    )

    ax.set_xticks(x, labels, size=9)
    if dataset_idx % 6 == 0:
        ax.set_yticks([0, 1])

    ax.tick_params(axis="y", labelsize=10)
    dataset_idx += 1

handles, labels = ax.get_legend_handles_labels()
fig.legend(
    handles, labels, loc="center", bbox_to_anchor=(0.5, -0.05), ncol=2, fontsize=11
)
fig.text(
    -0.005,
    0.5,
    "Relative Error (Normalized)",
    va="center",
    rotation="vertical",
    fontsize=11.2,
)
fig.tight_layout()
plt.show()

# save as pdf
fig.savefig("rq2_error_handling_scheme.pdf", bbox_inches="tight")

reduction_error_ls = (np.array(error_ls_ignore) - np.array(error_ls_rectify)) / (
    np.array(error_ls_ignore) + 0.0000000000000001
)
print(np.mean(reduction_error_ls), np.std(reduction_error_ls))